# Dados

In [7]:
import pandas as pd
import numpy as np
import random

path = 'Bases de dados/'
df = pd.read_csv(path + 'census.csv')

# Amostragem Sistemática

In [9]:
def amostragem_sistematica(dataset, amostras):
    intervalo = len(dataset)//amostras
    random.seed(1)
    inicio = random.randint(0, intervalo)
    indices = np.arange(inicio, len(dataset), step = intervalo)
    amostra_sistematica = dataset.iloc[indices]
    return amostra_sistematica

In [10]:
amostragem_sistematica(df, 100)

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
68,49,Self-emp-inc,191681,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,>50K
393,34,State-gov,98101,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,7688,0,45,?,>50K
718,22,Private,214399,Some-college,10,Never-married,Sales,Own-child,White,Female,0,0,15,United-States,<=50K
1043,44,Private,167005,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,7688,0,60,United-States,>50K
1368,52,Private,152234,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,Asian-Pac-Islander,Male,99999,0,40,Japan,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30943,33,Private,48010,Some-college,10,Divorced,Exec-managerial,Unmarried,White,Female,0,0,40,United-States,<=50K
31268,43,Private,306440,Prof-school,15,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,66,France,<=50K
31593,37,Private,171968,Assoc-voc,11,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States,<=50K
31918,26,Private,154571,Some-college,10,Never-married,Machine-op-inspct,Not-in-family,Asian-Pac-Islander,Male,0,0,45,United-States,>50K


# Amostragem por grupos

In [19]:
def amostragem_agrupamento(dataset, numero_grupos):
    intervalo = len(dataset) / numero_grupos
    grupos = []
    id_grupo = 0
    contagem = 0
    for _ in dataset.iterrows():
        grupos.append(id_grupo)
        contagem += 1
        if contagem > intervalo:
            contagem = 0
            id_grupo += 1

    dataset['grupo'] = grupos
    random.seed(1)
    grupo_selecionado = random.randint(0, numero_grupos)
    return dataset[dataset['grupo']== grupo_selecionado]


In [20]:
df_amostra_agrupamento = amostragem_agrupamento(df, 100)
df_amostra_agrupamento.shape, df_amostra_agrupamento['grupo'].value_counts()

((326, 16),
 grupo
 17    326
 Name: count, dtype: int64)

# Amostragem Estratificada

In [21]:
from sklearn.model_selection import StratifiedShuffleSplit


In [23]:
df['income'].value_counts(normalize=True)

income
<=50K    0.75919
>50K     0.24081
Name: proportion, dtype: float64

In [34]:
amostras = 100
split = StratifiedShuffleSplit(test_size=amostras / len(df))
for x,y in split.split(df,df['income']):
    df_x = df.iloc[x]
    df_y = df.iloc[y]


In [35]:
df_x.shape, df_y.shape

((32461, 16), (100, 16))

In [37]:
df_y['income'].value_counts()

income
<=50K    76
>50K     24
Name: count, dtype: int64

# Amostragem de reservatório

In [38]:
def amostragem_reservatorio(dataset, amostras):
    stream = []
    for i in range(len(dataset)):
        stream.append(i)

    i = 0
    tamanho = len(dataset)

    reservatorio = [0] * amostras
    for i in range(amostras):
        reservatorio[i] = stream[i]

    while i < tamanho:
        j = random.randrange(i + 1)
        if j < amostras:
            reservatorio[j] = stream[i]
        i += 1

    return dataset.iloc[reservatorio]


In [40]:
df_amostra_reservatorio = amostragem_reservatorio(df, 100)
df_amostra_reservatorio.shape

(100, 16)

In [41]:
df_amostra_reservatorio.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income,grupo
28106,22,Private,221480,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Female,0,0,40,United-States,<=50K,86
310,26,Private,59306,Bachelors,13,Never-married,Sales,Not-in-family,White,Male,0,0,40,United-States,<=50K,0
2845,50,Private,121685,Some-college,10,Married-civ-spouse,Sales,Husband,White,Male,0,0,45,United-States,>50K,8
27135,61,Private,270056,HS-grad,9,Divorced,Adm-clerical,Not-in-family,Asian-Pac-Islander,Female,0,0,40,Japan,<=50K,83
26804,44,Private,33155,Doctorate,16,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,45,United-States,<=50K,82
